In [57]:
import pandas as pd
from replace_with_warning import replace_with_warning
from pandas_helper import broadcast_simple
import numpy as np

from res_ind_lib import average_over_rp

In [58]:
iso3_to_wb = pd.read_csv("inputs/iso3_to_wb_name.csv", index_col="iso3", squeeze=True)


#Names to WB names
any_to_wb = pd.read_csv("inputs/any_name_to_wb_name.csv",index_col="any",squeeze=True)


# AAL

## agg data

In [59]:
gar_aal_data = pd.read_csv("inputs/GAR15 results feb 2016_AALmundo.csv", encoding="latin-1", thousands=",", )

# .drop(["ISO","Country"],axis=1)

#these are part of france
gar_aal_data.ISO.replace(["GUF", "GLP", "MTQ", "MYT", "REU"],"FRA", inplace=True)

#these ara part of the UK
gar_aal_data.ISO.replace(["FLK", "GIB", "MSR"],"GBR", inplace=True)

#wb spellings
gar_aal_data = gar_aal_data.set_index(replace_with_warning(gar_aal_data.ISO,iso3_to_wb)).drop(["ISO","Country"],axis=1)
gar_aal_data.index.name="country"


#aggregates UK and France pieces to one country
gar_aal_data =  gar_aal_data.sum(level="country") 


#takes exposed value out
gar_exposed_value = gar_aal_data.pop("EXPOSED VALUE")


#rename hazards
gar_aal_data = gar_aal_data.rename(columns=lambda s:(s.lower()))
gar_aal_data= gar_aal_data.rename(columns = {"tropical cyclones":"cyclones", "riverine floods":"flood", "storm surge":"surge"})


gar_aal_data

AAL = (gar_aal_data.T/gar_exposed_value).T
AAL.head()

gar_aal_data.sum().sum()

C:\Users\Admin\Box Sync\PYTHON - RESILIENCE\resilience_indicator_multihazard\replace_with_warning.py:21: UserWarning: These entries were not found in the dictionary:
ESH
  warnings.warn("These entries were not found in the dictionary:\n"+joiner.join(series_in[are_missing].unique()))


293387.32000000007

## wind and surge

In [60]:

def get_dk_over_k_from_file(path="inputs/GAR_data_surge.csv"):
    gardata_surge = pd.read_csv(path).dropna(axis=1, how="all")

    #these are part of france
    gardata_surge.Country.replace(["GUF", "GLP", "MTQ", "MYT", "REU"],"FRA", inplace=True)

    #these ara part of the UK
    gardata_surge.Country.replace(["FLK", "GIB", "MSR"],"GBR", inplace=True)


    gardata_surge = gardata_surge.set_index(replace_with_warning(gardata_surge.Country, iso3_to_wb))[["AAL", "Exposed Value (from GED)"]] # drop(["Country","Country name", "Region", "GA", "Hazard"],axis=1)

    # gardata_surge.rename(columns=lambda s: float(s[4:-5]))

    return gardata_surge["AAL"].sum(level=0)/gardata_surge["Exposed Value (from GED)"].sum(level=0)


aal_surge = get_dk_over_k_from_file("inputs/GAR_data_surge.csv")
aal_wind = get_dk_over_k_from_file("inputs/GAR_data_wind.csv")
aal_recomputed =aal_surge +aal_wind


((AAL.cyclones - aal_recomputed)/AAL.cyclones).abs().sort_values(ascending=False).head()




C:\Users\Admin\Box Sync\PYTHON - RESILIENCE\resilience_indicator_multihazard\replace_with_warning.py:21: UserWarning: These entries were not found in the dictionary:
ESH
  warnings.warn("These entries were not found in the dictionary:\n"+joiner.join(series_in[are_missing].unique()))


Costa Rica                        1.000000e+00
Tonga                             3.812629e-01
St. Vincent and the Grenadines    2.115747e-16
Venezuela, RB                     2.115004e-16
United States                     2.085513e-16
dtype: float64

In [61]:
c="Costa Rica"
f = (aal_surge/aal_recomputed).mean()
aal_surge.ix[c] = f*AAL.cyclones[c]
aal_wind.ix[c] = (1-f)*AAL.cyclones[c]


## AAL splitted

In [62]:
AAL_splitted = AAL.drop("cyclones",axis=1).assign(wind= aal_wind, surge = aal_surge).rename(columns=dict(floods="flood")).stack()
AAL_splitted.index.names=["country", "hazard"]

# PMLs and Exposed value

In [63]:
gardata =pd.read_csv("inputs/GAR15 results feb 2016_PML mundo.csv", encoding="latin-1", header=[0,1,2], index_col=0)#.sort_index(axis=1)
gardata.index.name = "country"

gardata = gardata.reset_index()

gardata.columns.names=["hazard","rp","what"]

#these are part of france
gardata.country = gardata.country.replace(["French Guiana", "Guadeloupe", "Martinique", "Mayotte", "Réunion"],"France")

#these ara part of the UK
gardata.country = gardata.country.replace(["Falkland Islands (Malvinas)", "Gibraltar", "Montserrat"],"United Kingdom")

gardata.country = replace_with_warning(gardata.country, any_to_wb.dropna())

gardata =gardata.set_index("country")

#aggregates UK and France
# gardata = gardata.sum(level="country", axis=1)


#Other format for zero
gardata = gardata.replace("---",0).astype(float)

#aggregates france and uk
gardata = gardata.sum(level="country")

gardata.head()


C:\Users\Admin\Box Sync\PYTHON - RESILIENCE\resilience_indicator_multihazard\replace_with_warning.py:21: UserWarning: These entries were not found in the dictionary:
Western Sahara
  warnings.warn("These entries were not found in the dictionary:\n"+joiner.join(series_in[are_missing].unique()))


hazard         EXPOSED VALUE  EARTHQUAKE                                      \
rp                  National          20                50               100   
what             million US$ million US$   [%] million US$   [%] million US$   
country                                                                        
Afghanistan         60187.90      317.83  0.53      637.57  1.06     1034.61   
Albania             40459.70      112.83  0.28      239.07  0.59      412.61   
Algeria            899206.00     2965.76  0.33     5719.85  0.64     8834.51   
American Samoa       1930.49        0.39  0.02        2.46  0.13        7.18   
Andorra              8381.65        0.19  0.00        0.39  0.00        0.64   

hazard                                                ...    Riverine floods  \
rp                           250               500    ...                100   
what             [%] million US$   [%] million US$    ...        million US$   
country                                               ...                      
Afghanistan     1.72     1800.74  2.99     2585.25    ...        1304.836191   
Albania         1.02      817.76  2.02     1308.38    ...         559.456815   
Algeria         0.98    14487.39  1.61    19624.90    ...        2455.765743   
American Samoa  0.37       21.09  1.09       39.03    ...                NaN   
Andorra         0.01        1.59  0.02        3.66    ...                NaN   

hazard                                                                  \
rp                                250                    500             
what                 [%]  million US$       [%]  million US$       [%]   
country                                                                  
Afghanistan     2.167938  1662.178063  2.761648  1749.645544  2.906972   
Albania         1.382751   985.990186  2.436969  1152.001339  2.847281   
Algeria         0.273104  5227.693307  0.581368  6507.867777  0.723735   
American Samoa       NaN          NaN       NaN          NaN       NaN   
Andorra              NaN          NaN       NaN          NaN       NaN   

hazard                                                        
rp                     1000                   1500            
what            million US$       [%]  million US$       [%]  
country                                                       
Afghanistan     1924.580505  3.197620  2099.515466  3.488268  
Albania         1342.639991  3.318463  1434.570014  3.545676  
Algeria         7652.708961  0.851052  8032.451716  0.893283  
American Samoa          NaN       NaN          NaN       NaN  
Andorra                 NaN       NaN          NaN       NaN  

[5 rows x 71 columns]

In [64]:

#Looks at the exposed value, for comparison
exposed_value_GAR = gardata.pop("EXPOSED VALUE").squeeze()
# df["exposed_value_GAR"].head()


#rename hazards
gardata = gardata.rename(columns=lambda s:(s.lower()))
gardata= gardata.rename(columns = {"wiind":"wind", "riverine floods":"flood", "storm surge":"surge"})

#rp to floats
def str_to_float(s):
    try:
        return (float(s))
    except ValueError:
        return s
gardata = gardata.rename(columns=str_to_float)

gardata.head()



#Asset losses per event
dK_gar = gardata.swaplevel("what","hazard",axis=1)["million us$"].swaplevel("rp","hazard",axis=1).stack(level=["hazard","rp"], dropna=False).sort_index()

#Exposed value per event
ev_gar=  broadcast_simple(exposed_value_GAR,dK_gar.index)

#Fraction of value destroyed
frac_value_destroyed_gar = dK_gar/ev_gar

frac_value_destroyed_gar.dropna().unstack("country").shape

frac_value_destroyed_gar.to_csv("intermediate/frac_value_destroyed_gar.csv", encoding="utf-8", header=True)

frac_value_destroyed_gar.head()

country      hazard      rp   
Afghanistan  earthquake  20.0     0.005281
                         50.0     0.010593
                         100.0    0.017190
                         250.0    0.029919
                         500.0    0.042953
dtype: float64

In [65]:
#Check. should be 0
(exposed_value_GAR - gar_exposed_value).abs().sort_values(ascending=True).head(5)

country
Afghanistan      0.0
New Caledonia    0.0
New Zealand      0.0
Nicaragua        0.0
Niger            0.0
dtype: float64

# Combines exposed value to frac destroyed from events

In [66]:
capital_losses_from_GAR_events = pd.read_csv("intermediate/capital_losses_from_GAR_events.csv", index_col=["country","hazard","rp"], squeeze=True)

In [67]:
frac_cap_distroyed_from_events = capital_losses_from_GAR_events/ev_gar.median(level="country")
frac_cap_distroyed_from_events.to_csv("intermediate/frac_cap_distroyed_from_events.csv", header=True)



In [68]:
(average_over_rp(frac_cap_distroyed_from_events).squeeze()/ AAL_splitted ).replace(np.inf,np.nan).dropna().sort_values()


country                   hazard    
France                    tsunami       0.241588
Cambodia                  wind          0.431152
Trinidad and Tobago       flood         0.435898
Micronesia, Fed. Sts.     surge         0.560576
Palau                     wind          0.587217
Pakistan                  wind          0.600110
Faeroe Islands            wind          0.601643
Oman                      wind          0.605902
                          surge         0.611014
Sri Lanka                 wind          0.619401
Bangladesh                wind          0.620611
New Zealand               wind          0.623906
French Polynesia          wind          0.626418
Sri Lanka                 surge         0.643248
Comoros                   wind          0.646975
Palau                     surge         0.647804
Papua New Guinea          wind          0.654225
Fiji                      wind          0.657703
France                    earthquake    0.657711
Comoros                   surge 

# adds event to PML to complete AAL  (old bad method)

In [69]:
frac_value_destroyed_gar.head()

country      hazard      rp   
Afghanistan  earthquake  20.0     0.005281
                         50.0     0.010593
                         100.0    0.017190
                         250.0    0.029919
                         500.0    0.042953
dtype: float64

In [70]:
AAL_splitted.head()

country      hazard    
Afghanistan  earthquake    0.002439
             tsunami       0.000000
             flood         0.001238
             surge         0.000000
             wind          0.000000
dtype: float64

 how much greater AAL is than sum of PML

In [71]:
paf= (AAL_splitted / average_over_rp(frac_value_destroyed_gar).squeeze()).replace(np.inf,np.nan).sort_values(ascending=False)
paf[paf>5]

country             hazard    
United States       earthquake    443.503685
China               earthquake     25.102987
Bangladesh          tsunami        19.450666
Philippines         surge          17.402475
Timor-Leste         tsunami        14.359563
China               surge          10.733380
Taiwan              surge           9.921706
Cyprus              tsunami         9.477856
Japan               surge           9.340137
Vanuatu             surge           6.538196
Qatar               earthquake      6.264938
Bahrain             earthquake      6.113478
Australia           surge           5.786772
Iran, Islamic Rep.  earthquake      5.665765
Vietnam             surge           5.532636
India               earthquake      5.485998
Guatemala           earthquake      5.108866
Peru                earthquake      5.098422
dtype: float64

In [72]:
paf.dropna().tail(30)

country           hazard 
Honduras          wind       1.543333
Japan             tsunami    1.542144
Timor-Leste       flood      1.537640
Guyana            flood      1.534599
Austria           flood      1.533728
Netherlands       flood      1.533283
Gambia, The       flood      1.530909
Puerto Rico       flood      1.528734
Denmark           flood      1.527811
Liberia           flood      1.525606
Jamaica           flood      1.525070
Japan             flood      1.523710
Georgia           flood      1.522316
Korea, Rep.       flood      1.521544
Grenada           wind       1.518630
Germany           flood      1.518202
Italy             flood      1.516215
Aruba             wind       1.506485
Faeroe Islands    wind       1.503113
Papua New Guinea  surge      1.497927
Switzerland       flood      1.494596
Guinea-Bissau     flood      1.490993
Albania           flood      1.481189
Belgium           flood      1.479016
Nicaragua         wind       1.478786
Yemen, Rep.       flood 

In [73]:
frac_value_destroyed_gar["Japan"]["flood"]*1e6

rp
20.0          6.464444
50.0        157.629328
100.0       912.188958
250.0      3356.835046
500.0      8810.043890
1000.0    15019.161836
1500.0    19445.214904
dtype: float64

## add frequent events

In [74]:
last_rp = 20
new_rp = 1

added_proba = 1/new_rp - 1/last_rp

new_frac_destroyed = (AAL_splitted - average_over_rp(frac_value_destroyed_gar).squeeze())/(added_proba)
   
#REMOVES 'tsunamis' and 'earthquakes' from this thing
# new_frac_destroyed = pd.DataFrame(new_frac_destroyed).query("hazard not in ['tsunami', 'earthquake']").squeeze()
new_frac_destroyed.describe(), new_frac_destroyed.sample(10)


C:\Users\Admin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


(count    976.000000
 mean       0.000480
 std        0.001673
 min        0.000000
 25%             NaN
 50%             NaN
 75%             NaN
 max        0.027633
 dtype: float64, country             hazard    
 Tuvalu              earthquake    0.000000
 Germany             earthquake    0.000090
 Iran, Islamic Rep.  surge         0.000000
 Romania             flood         0.000424
 Tanzania            flood         0.000529
 Moldova             tsunami       0.000000
 Togo                wind          0.000000
 Mongolia            flood         0.000355
 Philippines         surge         0.004448
 Tuvalu              surge         0.000000
 dtype: float64)

In [75]:
hop = frac_value_destroyed_gar.unstack()
hop[new_rp]=   new_frac_destroyed
hop= hop.sort_index(axis=1)
hop.head()


frac_value_destroyed_gar_completed = hop.stack()
frac_value_destroyed_gar_completed.head()  #this shows earthaquke was not updated

country      hazard      rp   
Afghanistan  earthquake  1.0      0.002005
                         20.0     0.005281
                         50.0     0.010593
                         100.0    0.017190
                         250.0    0.029919
dtype: float64

In [76]:
frac_value_destroyed_gar_completed.sample(10)

country         hazard   rp    
Sierra Leone    flood    20.0      0.024942
Myanmar         surge    20.0      0.001231
Tuvalu          tsunami  500.0     0.000000
Iraq            flood    500.0     0.049612
Algeria         tsunami  50.0      0.000000
Sudan           wind     20.0      0.000000
Mongolia        wind     1500.0    0.000000
Uzbekistan      wind     250.0     0.000000
Macedonia, FYR  surge    1000.0    0.000000
Azerbaijan      tsunami  250.0     0.000000
dtype: float64

### double check. expecint zeroes expect for quakes and tsunamis

In [77]:
(average_over_rp(frac_value_destroyed_gar_completed).squeeze() - AAL_splitted).abs().sort_values(ascending=False).sample(10)

country          hazard    
Grenada          earthquake    4.336809e-19
Burundi          surge         0.000000e+00
India            surge         0.000000e+00
                 wind          5.421011e-20
Burkina Faso     earthquake    0.000000e+00
Estonia          earthquake    1.694066e-21
Japan            earthquake    0.000000e+00
Gambia, The      wind          0.000000e+00
Swaziland        wind          0.000000e+00
Solomon Islands  earthquake    0.000000e+00
dtype: float64

### places where new values are higher than values for 20

In [78]:
test = frac_value_destroyed_gar_completed.unstack().replace(0,np.nan).dropna().assign(test=lambda x:x[new_rp]/x[20]).test

max_relative_exp = .8

overflow_frequent_countries = test[test>max_relative_exp].index
print("overflow in {n} (country, event)".format(n=len(overflow_frequent_countries)))

test[overflow_frequent_countries].sort_values(ascending=False)

overflow in 16 (country, event)


country               hazard    
United States         earthquake    43.952710
Senegal               earthquake     8.185474
Japan                 flood          3.473955
Netherlands           flood          3.446424
Chile                 tsunami        2.739425
Lebanon               flood          2.568359
China                 earthquake     2.307968
Luxembourg            flood          1.922135
Tonga                 tsunami        1.434211
Egypt, Arab Rep.      flood          1.293294
United Arab Emirates  flood          1.289766
Solomon Islands       tsunami        1.046737
Oman                  wind           1.019892
Philippines           surge          1.014407
Switzerland           flood          0.823777
Korea, Rep.           flood          0.822222
Name: test, dtype: float64

## for this places, add infrequent events


In [79]:
hop=frac_value_destroyed_gar_completed.unstack()
hop[1]=hop[1].clip(upper=max_relative_exp*hop[20])
frac_value_destroyed_gar_completed = hop.stack()

In [80]:
new_rp = 2000

added_proba = 1/2000

new_frac_destroyed = (AAL_splitted - average_over_rp(frac_value_destroyed_gar_completed).squeeze())/(added_proba)
   
#REMOVES 'tsunamis' and 'earthquakes' from this thing
# new_frac_destroyed = pd.DataFrame(new_frac_destroyed).query("hazard in ['tsunami', 'earthquake']").squeeze()
new_frac_destroyed.describe(), new_frac_destroyed.sample(10)

C:\Users\Admin\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


(count    9.760000e+02
 mean     4.388278e-03
 std      6.091275e-02
 min     -6.938894e-15
 25%               NaN
 50%               NaN
 75%               NaN
 max      1.786165e+00
 dtype: float64, country                hazard    
 Denmark                earthquake    0.000000e+00
 United Arab Emirates   wind          0.000000e+00
 Equatorial Guinea      surge         0.000000e+00
                        tsunami       0.000000e+00
 Morocco                flood         2.168404e-16
 Paraguay               wind          0.000000e+00
 Albania                flood         2.168404e-16
 Belize                 tsunami       0.000000e+00
 Tonga                  flood                  NaN
 Micronesia, Fed. Sts.  tsunami       1.553620e-02
 dtype: float64)

In [81]:
hop = frac_value_destroyed_gar.unstack()
hop[new_rp]=   new_frac_destroyed
hop= hop.sort_index(axis=1)
hop.head()


frac_value_destroyed_gar_completed = hop.stack()
frac_value_destroyed_gar_completed.head(10)  

country      hazard      rp    
Afghanistan  earthquake  20.0      5.280629e-03
                         50.0      1.059299e-02
                         100.0     1.718967e-02
                         250.0     2.991864e-02
                         500.0     4.295299e-02
                         1000.0    5.839446e-02
                         1500.0    6.763685e-02
                         2000.0   -8.673617e-16
             flood       20.0      9.722385e-03
                         50.0      1.736964e-02
dtype: float64

In [82]:
test = frac_value_destroyed_gar_completed.unstack().replace(0,np.nan).dropna().assign(test=lambda x:x[new_rp]/x[1500]).test
test[test>1.1].sort_values(ascending=False)

country               hazard    
United States         earthquake    8076.177552
China                 earthquake     281.390978
Philippines           surge          200.901584
Oman                  wind             8.603161
Lebanon               flood            4.892889
Senegal               earthquake       4.277388
Netherlands           flood            2.432692
Chile                 tsunami          2.372989
Luxembourg            flood            1.775911
Japan                 flood            1.688986
Egypt, Arab Rep.      flood            1.514697
United Arab Emirates  flood            1.425829
Name: test, dtype: float64

In [83]:
test[test>2].shape, test.shape

((8,), (223,))

In [84]:
frac_value_destroyed_gar_completed["United States"]["earthquake"]

rp
20.0      0.000006
50.0      0.000013
100.0     0.000019
250.0     0.000031
500.0     0.000042
1000.0    0.000054
1500.0    0.000064
2000.0    0.515774
dtype: float64

In [85]:
AAL_splitted.ix["United States"]*100

hazard
earthquake    0.026326
tsunami       0.000000
flood         0.010903
surge         0.015976
wind          0.036011
dtype: float64

In [86]:
pd.DataFrame((average_over_rp(frac_value_destroyed_gar).squeeze()/AAL_splitted).replace(0,np.nan).dropna().sort_values())

,,0
country,hazard,
United States,earthquake,0.002255
China,earthquake,0.039836
Bangladesh,tsunami,0.051412
Philippines,surge,0.057463
Timor-Leste,tsunami,0.069640
China,surge,0.093167
Taiwan,surge,0.100789
Cyprus,tsunami,0.105509
Japan,surge,0.107065
